In [1]:
import pickle, gzip, math, time, shutil, os, time, torch, matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

In [2]:
mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz,'rb') as f:
    ((x_train, y_train), (x_valid, y_valid),_) = pickle.load(f, encoding='latin1')

x_train, y_train, x_valid, y_valid = map(tensor, (x_train,y_train, x_valid, y_valid))

In [3]:
x_train.shape

torch.Size([50000, 784])

### Foundation Version

In [4]:
n, m = x_train.shape
c = y_train.max() + 1 
n, m,c

(50000, 784, tensor(10))

In [5]:
nh = 50

In [6]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)
b2 = torch.zeros(1)

In [7]:
def lin(x,w, b): 
    return x@w + b

In [9]:
t1 = lin(x_valid, w1, b1)
t1.shape

torch.Size([10000, 50])

In [10]:
def relu(x):
    return x.clamp_min(0.)

In [11]:
t = relu(t1)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [12]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [13]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

In [14]:
res.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [15]:
(res-y_valid).shape

torch.Size([10000, 10000])

In [16]:
res[:,0].shape

torch.Size([10000])

In [17]:
y_train, y_valid = y_train.float(), y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [18]:
def mse(output, target):
    return (output[:,0] - target).pow(2).mean()

In [19]:
mse(preds, y_train)

tensor(4308.76)

### Gradient and backward pass

In [20]:
def lin_grad(inp, out, w, b):
    # gradient of the matmul with respect to the input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum()
    b.g = out.g.sum(0)

In [21]:
def forward_and_backward(inp, targ):
    # Forward pass
    l1 = lin(inp,w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0] - targ
    loss = diff.pow(2).mean()
    
    # backward pass
    out.g = 2 * diff[:, None] / inp.shape[0]
    lin_grad (l2, out, w2, b2)
    l1.g = (l1>0).float() *l2.g
    lin_grad(inp,l1,w1,b1)

## Refactor the model
Layer as Class

In [22]:



class Relu():
    def __call__(self, inp) -> Any:
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [40]:
class Lin():
    def __init__(self, w, b):
        self.w = w 
        self.b = b
    
    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [41]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp 
        self.targ = targ
        self.out = mse(inp, targ)
        return self.out
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]
        

In [42]:
class Model:
    def __init__(self, w1, b1, w2,b2 ):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [43]:
model = Model(w1, b1, w2, b2)

In [44]:
loss = model(x_train, y_train)

In [45]:
model.backward()

In [46]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self):raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [58]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp > 0).float() * out.g

In [59]:
class Lin(Module):
    def __init__(self, w, b):
        self.w = w
        self.b = b
    def forward(self, inp):
        return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g 
        self.b.g = self.out.g.sum(0)
        

In [60]:
class Mse(Module):
    def forward(self, inp, targ):
        return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ):
        inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]
        

In [61]:
model = Model(w1, b1, w2, b2)

In [62]:
loss = model(x_train, y_train)

In [63]:
model.backward()

### Autograd

In [64]:
from torch import nn
import torch.nn.functional as F 

In [65]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out) -> None:
        super().__init__()
        self.w = torch.randn(n_in, n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp):
        return inp @ self.w + self.b 

In [68]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in, nh), nn.ReLU(), Linear(nh, n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:, None])

In [69]:
model = Model(m, nh, 1)

In [70]:
loss = model(x_train, y_train)

In [71]:
loss.backward()

In [72]:
l0 = model.layers[0]

In [73]:
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])